In [1]:
import numpy as np
import pandas as pd
import torch
import random

random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
train = pd.read_csv('x_train.csv',encoding='CP949')
test = pd.read_csv('x_test.csv',encoding='CP949')
train_label = pd.read_csv('y_train.csv',encoding='CP949')
submit = pd.read_csv('sample_submission.csv',encoding='CP949')

In [3]:
train

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [4]:
train_x = train.drop([], axis = 1) #해당 레이블은 각각 index. label 이므로 제거
test_x = test.drop([], axis = 1)
train_y = train_label

In [5]:
train_x_tensor = torch.FloatTensor(train_x.values).to(device)
train_y_tensor = torch.FloatTensor(train_y.values).to(device)
test_x_tensor = torch.FloatTensor(test_x.values).to(device)

In [6]:
train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=21486,
                                          shuffle=True,
                                          drop_last=True)

In [7]:
linear1 = torch.nn.Linear(14,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=0.3)

In [8]:
# Random Init => Xavier Init
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0478, -0.0664, -0.0263,  0.0546, -0.0122,  0.0989,  0.0697,  0.0884,
         -0.0712,  0.0008, -0.0303, -0.0247,  0.0359, -0.0238, -0.0525,  0.0529,
          0.0176, -0.0173, -0.1059,  0.0820, -0.0300,  0.0382, -0.0919,  0.0712,
          0.0200, -0.0219,  0.0987, -0.0520,  0.0550, -0.0699,  0.0100,  0.0577,
         -0.0198, -0.0307, -0.1005, -0.0753, -0.0817, -0.1036, -0.0291, -0.0063,
         -0.0795,  0.0733, -0.0351,  0.0163,  0.0487, -0.1022,  0.0059, -0.0286,
         -0.0523, -0.0159,  0.0311,  0.0563,  0.0603,  0.0464, -0.0843,  0.0204,
          0.0459, -0.0370, -0.0411, -0.0856,  0.0691,  0.0704,  0.0308,  0.0658,
          0.0844,  0.0180, -0.0320, -0.1024,  0.0244,  0.0904,  0.0029,  0.0489,
          0.0645, -0.0626, -0.0892, -0.0909, -0.0359, -0.0365,  0.0928, -0.0573,
         -0.0152, -0.0700, -0.0713,  0.0157, -0.0925,  0.1052,  0.0974, -0.0990,
         -0.0949, -0.1043, -0.0950,  0.0472,  0.0227,  0.0491,  0.0666, -0.0883,
      

In [9]:
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)

In [10]:
loss = torch.nn.MSELoss().to(device) #회귀 이므로 mseloss 사용
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [11]:

total_batch = len(data_loader)
model.train() # 주의사항 drop_out = True
for epoch in range(1000):
    avg_cost = 0

    for X, Y in data_loader:


        # 그래디언트 초기화
        optimizer.zero_grad()
        # Forward 계산
        hypothesis = model(X)
        # Error 계산
        cost = loss(hypothesis, Y)
        # Backparopagation
        cost.backward()
        # 가중치 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch
    if (epoch+1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0100 cost = 6168115.000000000
Epoch: 0200 cost = 805031.062500000
Epoch: 0300 cost = 606607.750000000
Epoch: 0400 cost = 427140.843750000
Epoch: 0500 cost = 375770.906250000
Epoch: 0600 cost = 379227.375000000
Epoch: 0700 cost = 371828.093750000
Epoch: 0800 cost = 329653.343750000
Epoch: 0900 cost = 304820.250000000
Epoch: 1000 cost = 308787.281250000
Learning finished


In [12]:
from sklearn.metrics import r2_score
with torch.no_grad():
    model.eval()

    prediction = model(test_x_tensor)
    train_prediction = model(train_x_tensor)
    accuracy = r2_score(train_y.교통량, train_prediction.detach().cpu())
    print('R2:', accuracy.item())

R2: 0.026160688964054768


In [13]:
predict = torch.argmax(prediction, dim = 1)

In [14]:
submit['predict'] = predict.detach().cpu().numpy()

In [15]:
submit.to_csv('2021_10_13_5.csv', index= False) # 실습 그대로 한 결과  epoch만 20 0.95370

1차 시도 : 실습 그대로 한 결과  epoch 20, dropout 0.5  score : 0.95370  
2차 시도 : 모든 레이어 파라미터 값 2배씩 증가후 코스트 기반으로 epcoh 4로 감소 score : 0.95760  
3차 시도 : 실습과 모두 같게하기 score : 0.96840  
4차 시도 : 3차시도 중 cost가 가장 낮았던 epoch 4까지 진행 score : 0.96690  
5차 시도 : 3차시도에서 epoch 100으로 증가 score : 0.12100
6차 시도 : 배치 사이즈 1000 및 epoch 30으로 진행 : 0.97810  
7차 시도 : 배치 사이즈 2000 및 epoch 30으로 진행 : 0.97370  
train best score : 0.99924

=> 0.981333 일 경우 0.97090이 나옴 :baseline